In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
csvData = pd.read_csv('./data/UrbanSound8K/metadata/UrbanSound8K.csv')
print(csvData.iloc[0, :])

slice_file_name    100032-3-0-0.wav
fsID                         100032
start                             0
end                        0.317551
salience                          1
fold                              5
classID                           3
class                      dog_bark
Name: 0, dtype: object


In [4]:
import IPython.display as ipd
ipd.Audio('./data/UrbanSound8K/audio/fold1/108041-9-0-5.wav')


In [5]:
class UrbanSoundDataset(Dataset):
#rapper for the UrbanSound8K dataset
    # Argument List
    #  path to the UrbanSound8K csv file
    #  path to the UrbanSound8K audio files
    #  list of folders to use in the dataset

    def __init__(self, csv_path, file_path, folderList):
        csvData = pd.read_csv(csv_path)
        #initialize lists to hold file names, labels, and folder numbers
        self.file_names = []
        self.labels = []
        self.folders = []
        #loop through the csv entries and only add entries from folders in the folder list
        for i in range(0,len(csvData)):
            if csvData.iloc[i, 5] in folderList:
                self.file_names.append(csvData.iloc[i, 0])
                self.labels.append(csvData.iloc[i, 6])
                self.folders.append(csvData.iloc[i, 5])

        self.file_path = file_path
        self.mixer = torchaudio.transforms.DownmixMono() #UrbanSound8K uses two channels, this will convert them to one
        self.folderList = folderList

    def __getitem__(self, index):
        #format the file path and load the file
        path = self.file_path + "fold" + str(self.folders[index]) + "/" + self.file_names[index]
        sound = torchaudio.load(path, out = None, normalization = True)
        #load returns a tensor with the sound data and the sampling frequency (44.1kHz for UrbanSound8K)
        soundData = self.mixer(sound[0])
        #downsample the audio to ~8kHz
        tempData = torch.zeros([160000, 1]) #tempData accounts for audio clips that are too short
        if soundData.numel() < 160000:
            tempData[:soundData.numel()] = soundData[:]
        else:
            tempData[:] = soundData[:160000]

        soundData = tempData
        soundFormatted = torch.zeros([32000, 1])
        soundFormatted[:32000] = soundData[::5] #take every fifth sample of soundData
        soundFormatted = soundFormatted.permute(1, 0)
        return soundFormatted, self.labels[index]

    def __len__(self):
        return len(self.file_names)

In [6]:
csv_path = './data/UrbanSound8K/metadata/UrbanSound8K.csv'
file_path = './data/UrbanSound8K/audio/'

train_set = UrbanSoundDataset(csv_path, file_path, range(1,10))
test_set = UrbanSoundDataset(csv_path, file_path, [10])
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 128, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 128, shuffle = True, **kwargs)

Train set size: 7895
Test set size: 837


In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, 80, 4)
        self.bn1 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(128, 128, 3)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(128, 256, 3)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(256, 512, 3)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(4)
        self.avgPool = nn.AvgPool1d(30) #input should be 512x30 so this outputs a 512x1
        self.fc1 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = self.avgPool(x)
        x = x.permute(0, 2, 1) #change the 512x1 to 1x512
        x = self.fc1(x)
        return F.log_softmax(x, dim = 2)

model = Net()
model.to(device)
print(model)

Net(
  (conv1): Conv1d(1, 128, kernel_size=(80,), stride=(4,))
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(256, 512, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (avgPool): AvgPool1d(kernel_size=(30,), stride=(30,), 

In [8]:
optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

In [9]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
        output = output.permute(1, 0, 2) #original output dimensions are batchSizex1x10
        loss = F.nll_loss(output[0], target) #the loss functions expects a batchSizex10 input
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [10]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        output = output.permute(1, 0, 2)
        pred = output.max(2)[1] # get the index of the max log-probability
        correct += pred.eq(target).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
log_interval = 20
for epoch in range(1, 41):
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
    scheduler.step()
    train(model, epoch)
    test(model, epoch)

Train Epoch: 1 [0/7895 (0%)]	Loss: 2.291376
Train Epoch: 1 [2560/7895 (32%)]	Loss: 2.264315
Train Epoch: 1 [5120/7895 (65%)]	Loss: 2.289759
Train Epoch: 1 [7680/7895 (97%)]	Loss: 2.219373

Test set: Accuracy: 100/837 (12%)

Train Epoch: 2 [0/7895 (0%)]	Loss: 2.220451
Train Epoch: 2 [2560/7895 (32%)]	Loss: 2.304013
Train Epoch: 2 [5120/7895 (65%)]	Loss: 2.255344
Train Epoch: 2 [7680/7895 (97%)]	Loss: 2.233193

Test set: Accuracy: 32/837 (4%)

Train Epoch: 3 [0/7895 (0%)]	Loss: 2.208180
Train Epoch: 3 [2560/7895 (32%)]	Loss: 2.239907
Train Epoch: 3 [5120/7895 (65%)]	Loss: 2.250428
Train Epoch: 3 [7680/7895 (97%)]	Loss: 2.283195

Test set: Accuracy: 32/837 (4%)

Train Epoch: 4 [0/7895 (0%)]	Loss: 2.245297
Train Epoch: 4 [2560/7895 (32%)]	Loss: 2.257275
Train Epoch: 4 [5120/7895 (65%)]	Loss: 2.227945
Train Epoch: 4 [7680/7895 (97%)]	Loss: 2.237137

Test set: Accuracy: 32/837 (4%)

Train Epoch: 5 [0/7895 (0%)]	Loss: 2.233145
Train Epoch: 5 [2560/7895 (32%)]	Loss: 2.224629
Train Epoch: 5 [51

Train Epoch: 37 [2560/7895 (32%)]	Loss: 2.206464
Train Epoch: 37 [5120/7895 (65%)]	Loss: 2.142814
Train Epoch: 37 [7680/7895 (97%)]	Loss: 2.244540

Test set: Accuracy: 32/837 (4%)

Train Epoch: 38 [0/7895 (0%)]	Loss: 2.176103
Train Epoch: 38 [2560/7895 (32%)]	Loss: 2.207323
Train Epoch: 38 [5120/7895 (65%)]	Loss: 2.211939
Train Epoch: 38 [7680/7895 (97%)]	Loss: 2.147498

Test set: Accuracy: 32/837 (4%)

Train Epoch: 39 [0/7895 (0%)]	Loss: 2.170258
Train Epoch: 39 [2560/7895 (32%)]	Loss: 2.156995
Train Epoch: 39 [5120/7895 (65%)]	Loss: 2.169023
Train Epoch: 39 [7680/7895 (97%)]	Loss: 2.161491

Test set: Accuracy: 102/837 (12%)

Train Epoch: 40 [0/7895 (0%)]	Loss: 2.157548
Train Epoch: 40 [2560/7895 (32%)]	Loss: 2.191098
Train Epoch: 40 [5120/7895 (65%)]	Loss: 2.259294
Train Epoch: 40 [7680/7895 (97%)]	Loss: 2.187340

Test set: Accuracy: 32/837 (4%)

